In [1]:
!pip install transformers evaluate accelerate

  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
Using cached nvidia_cublas_

In [2]:
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.7.0
    Uninstalling fsspec-2025.7.0:
      Successfully uninstalled fsspec-2025.7.0
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.7.0 requires fsspec==2025.7.0, but you have fsspec 2025.3.0 which is incompatible.


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [4]:
from datasets import load_dataset

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.distributions as distributions

In [6]:
import evaluate
import random

In [7]:
from huggingface_hub import login

# Replace with your real token
login("hf_")

In [8]:
# --- Configuration ---
SMALL_LLM_NAME = "gpt2" # A small model
LARGE_LLM_NAME = "gpt2-large" # A larger model from the same family (for reward model)
DATASET_NAME = "super_glue"
TASK_NAME = "boolq" # Example SuperGLUE task
REWARD_MODEL_BATCH_SIZE = 8
MAX_SEQUENCE_LENGTH = 128 # Adjust based on your task and model

In [9]:
# --- Load Models and Tokenizer ---
tokenizer = AutoTokenizer.from_pretrained(SMALL_LLM_NAME)
# Add padding token if not present
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [10]:
# Policy Model (Small LLM)
policy_model = AutoModelForCausalLM.from_pretrained(SMALL_LLM_NAME)

In [11]:
# Reward Model (Larger LLM - used to provide feedback, could also be a separate fine-tuned model)
# Here, we use a larger LLM to generate text and then potentially use some metric
# or a simple heuristic based on its output or likelihoods as a "reward signal".
# In a true RLAIF setup, you'd have a separate Reward Model trained on human preferences.
# For this simplified example, we'll use the larger model's generation quality as a proxy.
reward_model = AutoModelForCausalLM.from_pretrained(LARGE_LLM_NAME)

In [12]:
# --- Load Dataset ---
dataset = load_dataset(DATASET_NAME, TASK_NAME)
train_dataset = dataset["train"]
# For simplicity, take a small subset for training
train_dataset = train_dataset.select(range(100))

README.md: 0.00B [00:00, ?B/s]

boolq/train-00000-of-00001.parquet:   0%|          | 0.00/3.85M [00:00<?, ?B/s]

boolq/validation-00000-of-00001.parquet:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

boolq/test-00000-of-00001.parquet:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9427 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3270 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3245 [00:00<?, ? examples/s]

In [13]:
# --- Tokenization Function ---
def tokenize_function(examples):
    # Adjust based on the specific SuperGLUE task structure
    if TASK_NAME == "boolq":
        # Example: Combine passage and question
        inputs = [f"Passage: {p} Question: {q}" for p, q in zip(examples["passage"], examples["question"])]
        # The labels (true/false) will be used to guide the reward
    else:
         # Default to using the text column if available
        if "text" in examples:
            inputs = examples["text"]
        elif "sentence" in examples:
             inputs = examples["sentence"]
        else:
             # Handle other task structures as needed
            inputs = [str(x) for x in examples.values()][0] # Fallback to the first column as string

    # Tokenize the inputs and targets (if applicable)
    model_inputs = tokenizer(inputs, max_length=MAX_SEQUENCE_LENGTH, truncation=True, padding="max_length", return_tensors="pt")

    # In RL, the "label" is not directly used for supervised training of the policy,
    # but can be used in the reward calculation (e.g., rewarding generations that lead to the correct answer)
    if "label" in examples:
        model_inputs["labels"] = examples["label"]

    return model_inputs

In [14]:
tokenized_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=train_dataset.column_names)
tokenized_dataset.set_format(type="torch")

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [15]:
# --- REINFORCE Training Loop ---
optimizer = optim.Adam(policy_model.parameters(), lr=0.1)

num_episodes = 50 # Reduce for faster testing
gamma = 0.99 # Discount factor (less relevant for short sequences, but good practice)

# Evaluation metric (example: accuracy for BoolQ, might need custom eval for generation tasks)
metric = evaluate.load("accuracy")

In [16]:
for episode in range(num_episodes):
    # Sample a data point from the dataset
    batch = tokenized_dataset[random.randint(0, len(tokenized_dataset) - 1)]
    input_ids = batch["input_ids"].unsqueeze(0)
    attention_mask = batch["attention_mask"].unsqueeze(0)
    labels = batch["labels"].unsqueeze(0) if "labels" in batch else None

    # Get policy model output (generates next token distribution)
    # We will generate a sequence of tokens for a response
    generation_output = policy_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=MAX_SEQUENCE_LENGTH + 20, # Generate slightly longer sequence
        num_return_sequences=1,
        output_scores=True, # Needed to calculate log probs manually if not using model's logit output directly
        return_dict_in_generate=True,
        pad_token_id=tokenizer.pad_token_id
    )
    generated_sequence = generation_output.sequences[0]
    transition_scores = generation_output.scores

    # Calculate log probabilities of the generated sequence under the policy
    # This requires careful handling of score dimensions and token IDs
    # A simplified approach for this example: treat the first generated token as the "action"
    # and the reward based on the *full* generated sequence.
    # For full sequence REINFORCE, you would sum log_probs over the sequence.

    # Simplified: Just take the log prob of the first generated token after the input
    # Find the index where generation starts
    generation_start_index = input_ids.shape[1]
    if generation_start_index >= generated_sequence.shape[0]:
         # Handle cases where the sequence is not longer than the input
        print("Warning: Generated sequence not longer than input. Skipping episode.")
        continue

    # Get the predicted token ID at the first generation step
    predicted_token_id = generated_sequence[generation_start_index]

    # Get the log probabilities for the *first* generated token from the scores
    # transition_scores is a tuple of tensors, one for each generated step
    first_step_scores = transition_scores[0] # Scores for the first generated token
    first_step_probs = torch.softmax(first_step_scores, dim=-1)
    m = distributions.Categorical(first_step_probs)
    log_prob_first_token = m.log_prob(predicted_token_id)

    # --- RLAIF Reward Calculation ---
    # Use the larger LLM (reward_model) to provide a reward signal.
    # This is the crucial RLAIF part. How to get a reward?
    # Options:
    # 1. Use a separate Reward Model trained on human preferences. (Ideal)
    # 2. Use the larger LLM's likelihood of the *correct* answer or a desired outcome
    #    given the generated text and the prompt.
    # 3. Use a simple metric (e.g., BLEU score, accuracy) if applicable to the task,
    #    calculated using the generated text and the ground truth label.

    # Simplified RLAIF Reward Example (using the larger model's output or likelihood):
    # Here, we'll generate with the large model and compare somehow, or just use a placeholder metric.
    # A more advanced approach would involve:
    # - Generating multiple responses from the small model.
    # - Using the large model or a separate RM to score/rank these responses.
    # - Using the scores/ranks as the reward signal.

    # For this example, let's use a placeholder reward based on whether the
    # generated sequence contains a token that might indicate the correct answer
    # (This is a highly simplified and task-specific heuristic)
    generated_text = tokenizer.decode(generated_sequence[generation_start_index:], skip_special_tokens=True)

    reward = 0 # Default reward
    if TASK_NAME == "boolq":
        # A very simple heuristic for BoolQ: does the generated text contain "yes" or "no"?
        # And is it consistent with the ground truth label?
        if labels is not None:
            true_label = labels.item() # 0 for False, 1 for True
            if ("yes" in generated_text.lower() and true_label == 1) or ("no" in generated_text.lower() and true_label == 0):
                 reward = 1.0
            elif ("yes" in generated_text.lower() and true_label == 0) or ("no" in generated_text.lower() and true_label == 1):
                 reward = -1.0 # Penalize incorrect answer

    # --- REINFORCE Update ---
    # Calculate the loss: - log_prob * reward
    # For sequence generation, the REINFORCE gradient is based on the sum of log probs * episode reward
    # In our simplified example, we only used the log prob of the first generated token.
    # A proper implementation would calculate the sum of log_probs of the whole sequence.
    # Let's stick to the simplified single-token log_prob for the basic REINFORCE structure.

    # For a single step/token: Loss = - log(pi(a_t|s_t)) * G_t
    # G_t is the cumulative discounted reward from time t. In this single "action" example, G_t is the episode reward.
    G = torch.tensor(reward)

    # Calculate the loss
    # We need to compute the log probabilities of the *generated* sequence
    # This requires feeding the generated sequence back through the policy model
    # and getting the log likelihoods. This is more complex than sampling and using scores directly.

    # Let's revert to a simpler REINFORCE logic using the policy's ability to score tokens.
    # Instead of `generate`, let's get the policy's likelihood for a *target* sequence or the generated one.
    # This requires restructuring the loop to get log_probs of the *sampled* tokens.

    # --- Revised REINFORCE Loop (Generating token by token and tracking log_probs) ---
    log_probs_episode = []
    generated_ids = input_ids.clone() # Start with the input prompt

    # Decide how many tokens to generate per episode
    num_generation_steps = 10 # Example: Generate 10 new tokens

    for step in range(num_generation_steps):
        outputs = policy_model(generated_ids)
        # Get logits for the last token in the sequence
        last_token_logits = outputs.logits[:, -1, :]
        probs = torch.softmax(last_token_logits, dim=-1)
        m = distributions.Categorical(probs)

        # Sample an action (next token)
        action_token = m.sample()

        # Store the log probability of the sampled action
        log_probs_episode.append(m.log_prob(action_token))

        # Append the sampled token to the generated sequence
        generated_ids = torch.cat([generated_ids, action_token.unsqueeze(-1)], dim=-1)

        # Optional: Check for end-of-sequence token and stop early
        if action_token.item() == tokenizer.eos_token_id:
            break

    # --- RLAIF Reward Calculation (using the full generated sequence) ---
    generated_text = tokenizer.decode(generated_ids[0][input_ids.shape[1]:], skip_special_tokens=True)

    reward = 0.0 # Default reward
    if TASK_NAME == "boolq" and labels is not None:
        true_label = labels.item()
        # A more robust reward would involve the large model.
        # Example (concept): Ask the large model if the generated text answers the question correctly.
        prompt_for_rm = f"Passage: {tokenizer.decode(input_ids[0], skip_special_tokens=True)} Generated Answer: {generated_text} Is the generated answer correct? Yes or No."

        # Tokenize the prompt for the reward model
        rm_input = tokenizer(prompt_for_rm, return_tensors="pt")
        # Move to device if using GPU
        if torch.cuda.is_available():
             rm_input = {k: v.to(policy_model.device) for k, v in rm_input.items()}


        # Generate response from the reward model (GPT2-Large)
        # Limit generation length to get a concise "Yes" or "No" answer
        rm_response = reward_model.generate(
            **rm_input,
            max_new_tokens=5, # Generate a few tokens to get "Yes" or "No"
            num_return_sequences=1,
            pad_token_id=tokenizer.pad_token_id, # Ensure padding token is handled
            eos_token_id=tokenizer.eos_token_id # Ensure EOS token is handled
        )

        rm_text = tokenizer.decode(rm_response[0][rm_input['input_ids'].shape[1]:], skip_special_tokens=True).strip().lower()

        # Evaluate reward based on the reward model's response
        if ("yes" in rm_text and true_label == 1) or ("no" in rm_text and true_label == 0):
            reward = 1.0
        elif ("yes" in rm_text and true_label == 0) or ("no" in rm_text and true_label == 1):
            reward = -1.0
        else:
            reward = -0.5 # Penalty for unclear or wrong format from RM
            # print(f"RM gave unclear response: '{rm_text}' for prompt: '{prompt_for_rm}'")


    # Calculate discounted rewards (G_t) - In REINFORCE, G_t is the cumulative reward from time t.
    # For a single episode with a single reward at the end, G_t is just the total reward for all steps.
    # If we had step-wise rewards, we would use discounting. With an episode-end reward,
    # the effective G_t for each step is the final episode reward.
    # G = [reward] * len(log_probs_episode) # All steps get the same reward

    # Calculate the loss for the episode
    # Loss = - sum(log(pi(a_t|s_t)) * G_t)
    # Since G_t is the same for all t in this case, Loss = - sum(log(pi(a_t|s_t))) * Reward
    loss = -torch.stack(log_probs_episode).sum() * torch.tensor(reward).to(policy_model.device)

    # Backpropagate and update the policy network
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if episode % 10 == 0:
        print(f"Episode {episode}, Loss: {loss.item():.4f}, Reward: {reward:.2f}")
        print(f"Generated Text: {generated_text}")

print("Training finished.")

Episode 0, Loss: -20.2279, Reward: -0.50
Generated Text: ? When will he go to induction in College?!
Episode 10, Loss: -0.0000, Reward: -0.50
Generated Text: CollegeCollegeCollegeCollegeCollegeCollegeCollegeCollegeCollegeCollege
Episode 20, Loss: 0.0000, Reward: 1.00
Generated Text: CollegeCollegeCollegeCollegeCollegeCollegeCollegeCollegeCollegeCollege
Episode 30, Loss: -0.0000, Reward: -0.50
Generated Text: CollegeCollegeCollegeCollegeCollegeCollegeCollegeCollegeCollegeCollege


KeyboardInterrupt: 

In [ ]:
# --- Optional: Evaluate the fine-tuned policy model ---
# You would need to define an evaluation function relevant to your SuperGLUE task.
# For BoolQ, you could generate responses for the test set and evaluate accuracy.

# print("\nEvaluating fine-tuned model...")
# test_dataset = dataset["validation"].select(range(50)) # Use a small validation set
# tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True, remove_columns=test_dataset.column_names)
# tokenized_test_dataset.set_format(type="torch")

# policy_model.eval() # Set model to evaluation mode
# predictions = []
# references = []

# with torch.no_grad():
#     for batch in tokenized_test_dataset:
#         input_ids = batch["input_ids"].unsqueeze(0)
#         attention_mask = batch["attention_mask"].unsqueeze(0)
#         labels = batch["labels"].unsqueeze(0) if "labels" in batch else None

#         # Generate response
#         generated_ids = policy_model.generate(
#             input_ids=input_ids,
#             attention_mask=attention_mask,
#             max_length=input_ids.shape[1] + 20, # Generate a short response
#             num_return_sequences=1,
#             pad_token_id=tokenizer.pad_token_id
#         )

#         generated_text = tokenizer.decode(generated_ids[0][input_ids.shape[1]:], skip_special_tokens=True)

#         # For BoolQ, try to extract 'yes' or 'no' and compare with the label
#         predicted_answer = None
#         if "yes" in generated_text.lower():
#             predicted_answer = 1 # True
#         elif "no" in generated_text.lower():
#             predicted_answer = 0 # False

#         if predicted_answer is not None and labels is not None:
#              predictions.append(predicted_answer)
#              references.append(labels.item())

# if predictions:
#     eval_results = metric.compute(predictions=predictions, references=references)
#     print(f"Evaluation Results: {eval_results}")
# else:
#     print("Could not evaluate. Check reward logic and generated text.")
